In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,GridSearchCV

from xgboost import XGBRegressor 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
# from sklearn.externals import joblib
import joblib

import warnings
warnings.filterwarnings('ignore')

In [18]:
### 一般我们机器学习的流程

# 一、加载数据
#processs_datas/train_process_datas_x.csv
datas_x = pd.read_csv('./processs_datas/train_process_datas_x.csv')
# print(datas_x.head())
# print(datas_x.info(verbose=True,null_counts=True))
# processs_datas/train_datas_Y.csv
datas_y = pd.read_csv('./processs_datas/train_datas_Y.csv')
# print(datas_y.head())
# print(datas_y.info())
# print(datas_y['happiness'].value_counts())

### 标签里面存在异常值 -8
# ## 一般情况下对于标签存在异常的数据需要删除，注意对应的X也要删除
# ## 先合并X,Y,再删除
# datas = pd.concat([datas_x,datas_y],axis=1)
# # print(datas.head())
# # print(datas.info(verbose=True,null_counts=True))
# datas.replace(-8,np.nan,inplace=True)
# datas.dropna(axis=0,how='any',inplace=True)
# # print(datas.shape)
# # print(datas['happiness'].value_counts(dropna=True))

## 这里我们可以考虑将-8替换成3（特殊情况特殊对待）
datas_y.replace(-8,3,inplace=True)
# print(datas_y.shape)
# print(datas_y['happiness'].value_counts())

In [19]:
# 二、数据清洗与预处理  （这里在dataprocess里面完成了）
# 删除ID字段
datas_x.drop(['id'],axis=1,inplace=True)

# 三、获取数据的特征属性X和目标属性Y

# 四、数据分割
x_train,x_test,y_train,y_test = train_test_split(datas_x,datas_y,test_size=0.2,random_state=11)
# print(x_train.shape)
# print(x_test.shape)

In [20]:
print(x_train.shape)
print(x_test.shape)

(6400, 138)
(1600, 138)


In [21]:
# 五、特征工程  
# 六、模型构建
model = XGBRegressor(n_estimators=100,max_depth=5,gamma=10.0)
# 七、模型训练
model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=10.0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=12, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

In [22]:
model.feature_importances_

array([0.        , 0.00571481, 0.01219193, 0.00485876, 0.        ,
       0.        , 0.00676607, 0.        , 0.02818899, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01551197, 0.        , 0.02503277,
       0.01438187, 0.07094149, 0.        , 0.        , 0.        ,
       0.        , 0.01400221, 0.01865187, 0.        , 0.        ,
       0.02021011, 0.        , 0.01931997, 0.        , 0.        ,
       0.        , 0.        , 0.01223092, 0.02575209, 0.        ,
       0.        , 0.        , 0.        , 0.02043409, 0.        ,
       0.        , 0.        , 0.01813575, 0.03875272, 0.08670937,
       0.        , 0.02117576, 0.        , 0.        , 0.01480069,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01650549, 0.09459621,
       0.01421185, 0.01257589, 0.        , 0.        , 0.     

In [23]:
# 八、模型评估
print('训练集R2:',model.score(x_train,y_train))
print('测试集R2:',model.score(x_test,y_test))
y_train_hat = model.predict(x_train)
y_test_hat = model.predict(x_test)
print('训练集MSE：',mean_squared_error(y_train,y_train_hat))
print('测试集MSE：',mean_squared_error(y_test,y_test_hat))

训练集R2: 0.35403764525861703
测试集R2: 0.2820671104722916
训练集MSE： 0.4335864442842868
测试集MSE： 0.4786997024148377


In [24]:
# 九、模型持久化
## 实际上对于最终我们保存的模型来说，我们可以把所有数据进行训练，再保存模型

model.fit(datas_x,datas_y)
print(model.score(datas_x,datas_y))
datas_y_hat = model.predict(datas_x)
print(mean_squared_error(datas_y,datas_y_hat))

save_dir_path = './model'
import os
if os.path.exists(save_dir_path):
    pass
else:
    os.makedirs(save_dir_path)

joblib.dump(model,save_dir_path+'/xgb.pkl')

0.35289350943068154
0.4337788472395472


['./model/xgb.pkl']